In [1]:
# Initial Python environment setup...
import numpy as np # linear algebra
import pandas as pd # CSV file I/O (e.g. pd.read_csv)
import os # reading the input files we have access to

print(os.listdir('../input'))

['sample_submission.csv', 'GCP-Coupons-Instructions.rtf', 'train.csv', 'test.csv']


In [2]:
# Read the CSV file into a DataFrame
df = pd.read_csv('../input/train.csv')

# Display the first few rows of the DataFrame to inspect the data
print(df.head())

                             key  fare_amount          pickup_datetime  \
0    2009-06-15 17:26:21.0000001          4.5  2009-06-15 17:26:21 UTC   
1    2010-01-05 16:52:16.0000002         16.9  2010-01-05 16:52:16 UTC   
2   2011-08-18 00:35:00.00000049          5.7  2011-08-18 00:35:00 UTC   
3    2012-04-21 04:30:42.0000001          7.7  2012-04-21 04:30:42 UTC   
4  2010-03-09 07:51:00.000000135          5.3  2010-03-09 07:51:00 UTC   

   pickup_longitude  pickup_latitude  dropoff_longitude  dropoff_latitude  \
0        -73.844311        40.721319         -73.841610         40.712278   
1        -74.016048        40.711303         -73.979268         40.782004   
2        -73.982738        40.761270         -73.991242         40.750562   
3        -73.987130        40.733143         -73.991567         40.758092   
4        -73.968095        40.768008         -73.956655         40.783762   

   passenger_count  
0                1  
1                1  
2                2  
3       

In [3]:
# Assuming df is your DataFrame
row_count = df.shape[0]

print("Number of rows:", row_count)


Number of rows: 55423856


In [4]:
# Specify the size of the random sample you want
sample_size = 150000  # Adjust this to the desired sample size

# Get a random sample from the DataFrame
random_sample = df.sample(n=sample_size)

# Print the random sample
print("Random Sample:")
print(random_sample)


Random Sample:
                                    key  fare_amount          pickup_datetime  \
35275929    2009-07-14 21:59:39.0000002         6.10  2009-07-14 21:59:39 UTC   
23659976  2010-05-25 23:23:00.000000110         8.10  2010-05-25 23:23:00 UTC   
19110511    2010-09-17 21:48:55.0000002         5.70  2010-09-17 21:48:55 UTC   
21805989    2014-07-05 05:49:20.0000001        57.33  2014-07-05 05:49:20 UTC   
14267926   2011-10-28 16:57:00.00000051         6.50  2011-10-28 16:57:00 UTC   
...                                 ...          ...                      ...   
31765792    2011-03-27 17:35:07.0000003        10.90  2011-03-27 17:35:07 UTC   
9399816     2012-02-21 10:21:27.0000005         7.70  2012-02-21 10:21:27 UTC   
53151792    2010-09-01 20:27:18.0000003         4.10  2010-09-01 20:27:18 UTC   
40090907  2012-01-30 19:02:00.000000192         4.90  2012-01-30 19:02:00 UTC   
52506326    2012-12-31 12:08:34.0000002        10.50  2012-12-31 12:08:34 UTC   

          pi

In [5]:
df = random_sample

In [6]:
# Display the number of missing values in each column
missing_values = df.isnull().sum()
print("Missing values per column:")
print(missing_values)

Missing values per column:
key                  0
fare_amount          0
pickup_datetime      0
pickup_longitude     0
pickup_latitude      0
dropoff_longitude    1
dropoff_latitude     1
passenger_count      0
dtype: int64


Basically my objective is identify the popular areas and over  popular areas for taxi activities and how these vary over time in here I removing the unwanted columns for analysis.

In [7]:
# Remove unnecessary columns for analysis
columns_to_keep = ['key', 'pickup_datetime', 'pickup_longitude', 'pickup_latitude', 'dropoff_longitude', 'dropoff_latitude']  # Add relevant column names
df = df[columns_to_keep]
print(df.head())

                                    key          pickup_datetime  \
35275929    2009-07-14 21:59:39.0000002  2009-07-14 21:59:39 UTC   
23659976  2010-05-25 23:23:00.000000110  2010-05-25 23:23:00 UTC   
19110511    2010-09-17 21:48:55.0000002  2010-09-17 21:48:55 UTC   
21805989    2014-07-05 05:49:20.0000001  2014-07-05 05:49:20 UTC   
14267926   2011-10-28 16:57:00.00000051  2011-10-28 16:57:00 UTC   

          pickup_longitude  pickup_latitude  dropoff_longitude  \
35275929        -73.949474        40.785228         -73.932937   
23659976        -73.981963        40.773023         -73.953395   
19110511        -73.964852        40.791478         -73.964925   
21805989        -73.968245        40.765059         -73.784073   
14267926        -73.971405        40.755967         -73.984503   

          dropoff_latitude  
35275929         40.795254  
23659976         40.779457  
19110511         40.791470  
21805989         40.643561  
14267926         40.757088  


In [8]:
# Drop rows with missing values
df = df.dropna()

In [9]:
df['pickup_datetime'] = pd.to_datetime(df['pickup_datetime'])

In [10]:
# Filter data for the year 2012
df_2012 = df[df['pickup_datetime'].dt.year == 2012]


# Print the number of rows in the filtered DataFrame
print("Number of Rows for the Year 2012:", len(df_2012))

Number of Rows for the Year 2012: 24240


In [11]:
# Define conditions for valid latitude and longitude values
valid_latitude_condition = ((df_2012['pickup_latitude'] != 0) & (df_2012['pickup_latitude'] >= 40.5) & (df_2012['pickup_latitude'] <= 40.9) &
                            (df_2012['dropoff_latitude'] != 0) & (df_2012['dropoff_latitude'] >= 40.5) & (df_2012['dropoff_latitude'] <= 40.9))
valid_longitude_condition = ((df_2012['pickup_longitude'] != 0) & (df_2012['pickup_longitude'] >= -74.2) & (df_2012['pickup_longitude'] <= -73.6) &
                             (df_2012['dropoff_longitude'] != 0) & (df_2012['dropoff_longitude'] >= -74.2) & (df_2012['dropoff_longitude'] <= -73.6))

# Apply the conditions using loc to avoid the warning
df_cleaned = df_2012.loc[valid_latitude_condition & valid_longitude_condition]

In [12]:
# Assuming df is your DataFrame
row_count = df_cleaned.shape[0]

print("Number of rows:", row_count)

Number of rows: 23579


In [13]:
# Save the filtered DataFrame to a CSV file
df_cleaned.to_csv('Spatial_sample_150000.csv', index=False)